In [1]:
import sys
import numpy as np
import pandas as pd
import time
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
tqdm.pandas()
sys.version

C:\Users\victo\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


'3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]'

In [4]:
df = pd.read_csv("data/PROSAIL_LUT.csv")
df = df.drop(df.columns[0],axis=1)
df

,n,cab,car,cbrown,cw,cm,lai,lidfa,lidfb,hspot,...,2491nm,2492nm,2493nm,2494nm,2495nm,2496nm,2497nm,2498nm,2499nm,2500nm
0,1.0,97.302,7.785,0.0,0.002227,0.004718,6.2295,-0.35,-0.15,0.01,...,0.050726,0.050480,0.050363,0.050237,0.050177,0.050236,0.049952,0.050094,0.049689,0.049967
1,1.0,76.338,39.123,0.0,0.006166,0.005498,8.9895,-0.35,-0.15,0.01,...,0.018476,0.018327,0.018248,0.018139,0.018094,0.018082,0.017943,0.017943,0.017785,0.017820
2,1.0,24.066,40.569,0.0,0.001918,0.004271,3.9815,-0.35,-0.15,0.01,...,0.058177,0.057921,0.057800,0.057673,0.057611,0.057679,0.057378,0.057539,0.057103,0.057413
3,1.0,66.426,13.497,0.0,0.004102,0.007857,6.8655,-0.35,-0.15,0.01,...,0.022507,0.022375,0.022310,0.022239,0.022206,0.022235,0.022086,0.022155,0.021948,0.022084
4,1.0,28.734,0.435,0.0,0.003725,0.004335,9.8885,-0.35,-0.15,0.01,...,0.035245,0.034999,0.034872,0.034703,0.034631,0.034625,0.034383,0.034410,0.034117,0.034217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,40.626,58.575,0.0,0.001001,0.001194,1.9235,-0.35,-0.15,0.01,...,0.111252,0.110905,0.110725,0.110485,0.110383,0.110368,0.110015,0.110049,0.109616,0.109760
9996,1.0,26.118,2.667,0.0,0.001324,0.006040,9.9905,-0.35,-0.15,0.01,...,0.056801,0.056642,0.056586,0.056587,0.056570,0.056734,0.056475,0.056788,0.056321,0.056839
9997,1.0,96.042,1.197,0.0,0.007375,0.004866,3.1725,-0.35,-0.15,0.01,...,0.015648,0.015519,0.015449,0.015350,0.015310,0.015291,0.015176,0.015162,0.015038,0.015047
9998,1.0,51.426,42.357,0.0,0.004763,0.002465,0.7955,-0.35,-0.15,0.01,...,0.027664,0.027529,0.027455,0.027346,0.027303,0.027264,0.027136,0.027097,0.026963,0.026946


In [5]:
def select_bands(df):
    sentinel_bands = [443, 490, 560, 665, 705, 740, 783, 842, 865, 945, 1375, 1610, 2190]
    sentinel_bands = [str(b)+"nm" for b in sentinel_bands]
    cols = list(df.columns[:(2119-(2501-400))])
    cols.extend(sentinel_bands)
    df = df[cols]
    return df, sentinel_bands

In [6]:
df, bands = select_bands(df)
df

,n,cab,car,cbrown,cw,cm,lai,lidfa,lidfb,hspot,...,665nm,705nm,740nm,783nm,842nm,865nm,945nm,1375nm,1610nm,2190nm
0,1.0,97.302,7.785,0.0,0.002227,0.004718,6.2295,-0.35,-0.15,0.01,...,0.012113,0.022132,0.261843,0.483119,0.483456,0.483524,0.480397,0.379877,0.319168,0.156530
1,1.0,76.338,39.123,0.0,0.006166,0.005498,8.9895,-0.35,-0.15,0.01,...,0.012266,0.029535,0.290071,0.499659,0.499876,0.499830,0.488893,0.295304,0.239899,0.099519
2,1.0,24.066,40.569,0.0,0.001918,0.004271,3.9815,-0.35,-0.15,0.01,...,0.015545,0.099201,0.350669,0.409184,0.409125,0.409081,0.407064,0.344106,0.299906,0.162125
3,1.0,66.426,13.497,0.0,0.004102,0.007857,6.8655,-0.35,-0.15,0.01,...,0.012216,0.034221,0.283431,0.430488,0.431105,0.431249,0.426873,0.301360,0.239502,0.095816
4,1.0,28.734,0.435,0.0,0.003725,0.004335,9.8885,-0.35,-0.15,0.01,...,0.014375,0.087648,0.409233,0.540446,0.540882,0.540934,0.532997,0.362480,0.300902,0.140563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,40.626,58.575,0.0,0.001001,0.001194,1.9235,-0.35,-0.15,0.01,...,0.013111,0.053931,0.229818,0.273797,0.273620,0.273551,0.272920,0.256407,0.245585,0.192992
9996,1.0,26.118,2.667,0.0,0.001324,0.006040,9.9905,-0.35,-0.15,0.01,...,0.015068,0.094259,0.391148,0.492213,0.493325,0.493656,0.492541,0.402936,0.325949,0.150500
9997,1.0,96.042,1.197,0.0,0.007375,0.004866,3.1725,-0.35,-0.15,0.01,...,0.012109,0.022088,0.225567,0.352640,0.352243,0.352057,0.346299,0.234160,0.198237,0.088380
9998,1.0,51.426,42.357,0.0,0.004763,0.002465,0.7955,-0.35,-0.15,0.01,...,0.014361,0.032839,0.113745,0.131782,0.132347,0.132513,0.132630,0.116327,0.109184,0.073054


In [7]:
rf = RandomForestRegressor()
scoring = ["r2", "neg_mean_squared_error", "neg_mean_absolute_percentage_error"]
scores = cross_validate(rf, df[bands], df[["lai"]].values.ravel(), scoring=scoring)

In [8]:
scores

{'fit_time': array([5.3323276 , 5.16124177, 5.14313436, 5.12240767, 5.16878843]),
 'score_time': array([0.05104136, 0.04306412, 0.04066396, 0.04905438, 0.05093455]),
 'test_r2': array([0.99365365, 0.99478681, 0.99548691, 0.99604964, 0.99342034]),
 'test_neg_mean_squared_error': array([-0.05242477, -0.04390068, -0.03744336, -0.0341435 , -0.05284586]),
 'test_neg_mean_absolute_percentage_error': array([-0.01214863, -0.01461719, -0.0113791 , -0.01182507, -0.01274926])}